In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from IES_Downloader import IES_Downloader

In [3]:
!pip install tqdm

# Lecture 6 - IES Web scraper

by Jan Šíla, based heavily on work of Vítek Macháček

April 1st, 2020

* Putting it all together
* OOP + Pandas + Requests + BeautifulSoup

Midterm April 7th as planned -> mainly scraping. More info to come

## Object-oriented Programming
* Brief reminder of how objects work

### Docstring + Objects + Inheriting + Constructors

In [4]:
class NoConstructorClasss:
    def meth(self):
        print('hi')
        
nc = NoConstructorClasss()
nnc = NoConstructorClasss()
nnnc = NoConstructorClasss()
nc.meth()
nnnc.meth()

hi
hi


In [5]:
class Parent:
    '''
    A very brief and helpful description of the Parent class
    '''
    def __init__(self,arg):
        '''
        Exact and brief description of parent's constructor
        '''
        self.attribute = 'x'
        self.id = arg

    def main_method(self):
        raise NotImplementedError("override this in child class")
    
    def parentMethod(self):
        '''
        Good description of parentMethod
        '''
        pass
    
class Child(Parent):
    '''
    This is what Child is good for
    '''
    def __init__(self,arg,desc):
        '''
        Exact and brief description of Child's contructor
        '''
        super().__init__(arg) #id is 999 in the example
        self.desc = desc
        self.id = 2 #id gets overridden here and set as 2
    
    def main_method(self):
        """ Overriding parent method """
        print('I am the main method!')
        
    def childMethod(self,arg):
        '''
        Good description of childMethod
        '''
        return len(arg)

    
par = Parent('A parent')
ch = Child(999,'A child')


In [6]:
ch.id

2

In [7]:
?ch

Type:           Child
String form:    <__main__.Child object at 0x11eec4c10>
Docstring:      This is what Child is good for
Init docstring: Exact and brief description of Child's contructor


In [8]:
par.main_method() #this should fail

NotImplementedError: override this in child class

In [9]:
ch.main_method() #this should work ok

I am the main method!


## Task:
* A parser of IES websites with following features:
    * All info about people from [Internal faculty](http://ies.fsv.cuni.cz/en/node/48), [External lecturers](http://ies.fsv.cuni.cz/en/node/49), [Ph.D. candidates](http://ies.fsv.cuni.cz/en/node/51) and [Administration](http://ies.fsv.cuni.cz/en/node/50)
    * All info about [all](http://ies.fsv.cuni.cz/en/node/109) theses between 1994 and 2019 won'be covered as we have problems with the website
    * Also all courses! But no list of courses available ...

### Robots.txt

* Is it OK to scrape?
* Guidance for search engines etc.


https://www.promptcloud.com/blog/how-to-read-and-respect-robots-file/

In [10]:
requests.get('http://ies.fsv.cuni.cz/robots.txt')

<Response [404]>

In [11]:
print(requests.get('http://sreality.cz/robots.txt').text)

User-agent: *
Disallow: /advertpdf/
Disallow: /favourites-info
Disallow: *_buri=
Disallow: /adresar/*page=
Disallow: /adresar/*perPage=
Disallow: /adresar/*search=
Disallow: /adresar/*letter=
Disallow: /adresar/*id=
Disallow: /firma/*page=
Disallow: /firma/*perPage=
Disallow: /firma/*search=
Disallow: /firma/*letter=
Disallow: /firma/*id=
Disallow: /hledani/*,
Allow: /hledani/*region=*,
Disallow: /rk-detail
Disallow: *bez-aukce=
Disallow: *without-auction=


User-agent: SeznamBot
Disallow: /advertpdf/
Disallow: /en/
Disallow: /ru/
Disallow: /favourites-info
Disallow: *_buri=
Disallow: /adresar/*page=
Disallow: /adresar/*perPage=
Disallow: /adresar/*search=
Disallow: /adresar/*letter=
Disallow: /adresar/*id=
Disallow: /firma/*page=
Disallow: /firma/*perPage=
Disallow: /firma/*search=
Disallow: /firma/*letter=
Disallow: /firma/*id=
Disallow: /hledani/*,
Allow: /hledani/*region=*,
Disallow: /rk-detail
Disallow: *bez-aukce=
Disallow: *without-auction=

Sitemap: https://www.sreality.cz/site

## Pages

### Find all persons?
[Current faculty](http://ies.fsv.cuni.cz/en/node/48)

1. understand structure of the website

In [2]:
def getSoup(link):
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml')

In [13]:
bf = getSoup('https://ies.fsv.cuni.cz/en/node/48')

In [14]:
tds = bf.findAll('td', {'class':'peopleTableCellName'})

In [15]:
def getAllLinks(link):
    soup = getSoup(link)
    tds = soup.findAll('td', {'class':'peopleTableCellName'})
    return ['https://ies.fsv.cuni.cz' + td.find('a')['href'] for td in tds]

links = getAllLinks('http://ies.fsv.cuni.cz/en/node/48')
links

['https://ies.fsv.cuni.cz/en/staff/barunik',
 'https://ies.fsv.cuni.cz/en/staff/bauerm',
 'https://ies.fsv.cuni.cz/en/staff/baxajaromir',
 'https://ies.fsv.cuni.cz/en/staff/bertoli',
 'https://ies.fsv.cuni.cz/en/staff/antosova',
 'https://ies.fsv.cuni.cz/en/staff/cahlik',
 'https://ies.fsv.cuni.cz/en/staff/fcech',
 'https://ies.fsv.cuni.cz/en/staff/cervinka',
 'https://ies.fsv.cuni.cz/en/staff/chytilova',
 'https://ies.fsv.cuni.cz/en/staff/dedek',
 'https://ies.fsv.cuni.cz/en/staff/dolezalova',
 'https://ies.fsv.cuni.cz/en/staff/gersl',
 'https://ies.fsv.cuni.cz/en/staff/gregor',
 'https://ies.fsv.cuni.cz/en/staff/havranek',
 'https://ies.fsv.cuni.cz/en/staff/irsova',
 'https://ies.fsv.cuni.cz/en/staff/hlavacekm',
 'https://ies.fsv.cuni.cz/en/staff/hollmannov',
 'https://ies.fsv.cuni.cz/en/staff/holub',
 'https://ies.fsv.cuni.cz/en/staff/horvath',
 'https://ies.fsv.cuni.cz/en/staff/jakubik',
 'https://ies.fsv.cuni.cz/en/staff/janda',
 'https://ies.fsv.cuni.cz/en/staff/jansky',
 'https:

## Person's characteristics?

In [16]:
def getName(link):
    soup = getSoup(link)
    return soup.find('h2').text
names = [getName(link) for link in links]

In [19]:
bf = getSoup(links[0])

In [23]:
bf.find('h2')

<h2>doc. PhDr. Jozef Baruník Ph.D.</h2>

In [127]:
"http://ies.fsv.cuni.cz/en/staff/barunik".split('/')[-1]

'barunik'

In [43]:
def getID(link):
    #http://ies.fsv.cuni.cz/en/staff/barunik
    return link.split('/')[-1]
[getID(link) for link in links]

['barunik',
 'bauerm',
 'baxajaromir',
 'bertoli',
 'antosova',
 'cahlik',
 'fcech',
 'cervinka',
 'chytilova',
 'dedek',
 'dolezalova',
 'gersl',
 'gregor',
 'havranek',
 'irsova',
 'hlavacekm',
 'hollmannov',
 'holub',
 'horvath',
 'jakubik',
 'janda',
 'jansky',
 'kemenyova',
 'kocenda',
 'komarek',
 'kristoufek',
 'kukacka',
 'maresj',
 'mejstrik',
 'nevrla',
 'novakji',
 'opatrny',
 'malirova',
 'palansky',
 'paulus',
 'neprasova',
 'gebicka',
 'pinter',
 'scasny',
 'schneider',
 'schwarz',
 'sedivy',
 'semerak',
 'reckova',
 'gregorovalenka',
 'reichlova',
 'teply',
 'vacek',
 'vacha',
 'visek',
 'vosvrda',
 'prochazkova',
 'zacek']

In [24]:
links[:2]

['https://ies.fsv.cuni.cz/en/staff/barunik',
 'https://ies.fsv.cuni.cz/en/staff/bauerm']

In [25]:
bf = getSoup(links[0])

In [35]:
node = bf.find('strong',text='Phone:')

In [38]:
node.next_sibling.strip()

'+420(776)259273'

In [40]:
def getNextSiblingOfStrong(link,characteristic):
    soup = getSoup(link)
    strong = soup.find('strong',text=characteristic)
    return strong.next_sibling.strip()

def getMoreCharacteristics(link, characteristics):
    return [getNextSiblingOfStrong(link,char) for char in characteristics]



[getMoreCharacteristics(link,['Phone:','Office:','Position:']) for link in links[:2]]

[['+420(776)259273', '503', 'Associate Professor'],
 ['222 112 329', '402', 'Associate Professor']]

* Let's do an object!

In [41]:
class Person:
    def __init__(self,link):
        self.soup = getSoup(link)
        self.office = self.getNextSiblingOfStrong('Office:')
        self.phone = self.getNextSiblingOfStrong('Phone:')
        
    def getNextSiblingOfStrong(self, characteristic):
        strong = self.soup.find('strong',text=characteristic)
        return strong.next_sibling

people = [Person(link) for link in links[:2]]

In [42]:
people

[<__main__.Person at 0x121327e90>, <__main__.Person at 0x121321150>]

In [45]:
#forgot to add an attribute to the constructor?
[p.soup.find('h2').text for p in people]

['doc. PhDr. Jozef Baruník Ph.D.', 'doc. PhDr. Michal Bauer Ph.D.']

In [48]:
for p_instance in people:
    p_instance.full_name = p_instance.soup.find('h2').text

In [50]:
people[0].full_name

'doc. PhDr. Jozef Baruník Ph.D.'

In [51]:
Person(links[5]).full_name

AttributeError: 'Person' object has no attribute 'full_name'

In [4]:
bf = getSoup('https://ies.fsv.cuni.cz/content/tree/index/lang/en')

In [5]:
int(bf.find('a',text='Scheduled shutdown of servers').parent.parent.parent.find('span').text)

25

In [9]:
[el.parent.parent.parent.find('span').text for el in bf.find_all('a',text='Scheduled shutdown of servers')]

['25', '26']

In [57]:
[event.findAll('span') for event in table]

[[<span>6</span>, <span>25</span>, <span>26</span>, <span>30</span>]]

In [10]:
type(5)

int